## Graph Convolutional Netoworks and the ACM Heterogenous Graph
this is a slightly modified version of the notebook from the DGL tutorial
https://docs.dgl.ai/en/0.4.x/tutorials/hetero/1_basics.html


In [1]:
%matplotlib inline


.. currentmodule:: dgl

Working with Heterogeneous Graphs
=================================

**Author**: Quan Gan, `Minjie Wang <https://jermainewang.github.io/>`_, Mufei Li,
George Karypis, Zheng Zhang

In this tutorial, you learn about:

* Examples of heterogenous graph data and typical applications.

* Creating and manipulating a heterogenous graph in DGL.

* Implementing `Relational-GCN <https://arxiv.org/abs/1703.06103>`_, a popular GNN model,
  for heterogenous graph input.

* Training a model to solve a node classification task.

Heterogeneous graphs, or *heterographs* for short, are graphs that contain
different types of nodes and edges. The different types of nodes and edges tend
to have different types of attributes that are designed to capture the
characteristics of each node and edge type. Within the context of
graph neural networks, depending on their complexity, certain node and edge types
might need to be modeled with representations that have a different number of dimensions.

DGL supports graph neural network computations on such heterogeneous graphs, by
using the heterograph class and its associated API.


Examples of heterographs
-----------------------
Many graph datasets represent relationships among various types of entities.
This section provides an overview for several graph use-cases that show such relationships 
and can have their data represented as heterographs.

Citation graph 
~~~~~~~~~~~~~~~
The Association for Computing Machinery publishes an `ACM dataset <https://aminer.org/citation>`_ that contains two
million papers, their authors, publication venues, and the other papers
that were cited. This information can be represented as a heterogeneous graph.

The following diagram shows several entities in the ACM dataset and the relationships among them 
(taken from `Shi et al., 2015 <https://arxiv.org/pdf/1511.04854.pdf>`_).

.. figure:: https://data.dgl.ai/tutorial/hetero/acm-example.png# 

This graph has three types of entities that correspond to papers, authors, and publication venues.
It also contains three types of edges that connect the following:

* Authors with papers corresponding to *written-by* relationships

* Papers with publication venues corresponding to *published-in* relationships

* Papers with other papers corresponding to *cited-by* relationships


Recommender systems 
~~~~~~~~~~~~~~~~~~~~ 
The datasets used in recommender systems often contain
interactions between users and items. For example, the data could include the
ratings that users have provided to movies. Such interactions can be modeled
as heterographs.

The nodes in these heterographs will have two types, *users* and *movies*. The edges
will correspond to the user-movie interactions. Furthermore, if an interaction is
marked with a rating, then each rating value could correspond to a different edge type.
The following diagram shows an example of user-item interactions as a heterograph.

.. figure:: https://data.dgl.ai/tutorial/hetero/recsys-example.png


Knowledge graph 
~~~~~~~~~~~~~~~~
Knowledge graphs are inherently heterogenous. For example, in
Wikidata, Barack Obama (item Q76) is an instance of a human, which could be viewed as
the entity class, whose spouse (item P26) is Michelle Obama (item Q13133) and
occupation (item P106) is politician (item Q82955). The relationships are shown in the following.
diagram.

.. figure:: https://data.dgl.ai/tutorial/hetero/kg-example.png




Creating a heterograph in DGL
-----------------------------
You can create a heterograph in DGL using the :func:`dgl.heterograph` API.
The argument to :func:`dgl.heterograph` is a dictionary. The keys are tuples
in the form of ``(srctype, edgetype, dsttype)`` specifying the relation name
and the two entity types it connects. Such tuples are called *canonical edge
types*. The values are data to initialize the graph structures, that is, which
nodes the edges actually connect.

For instance, the following code creates the user-item interactions heterograph shown earlier.



In [2]:
# Each value of the dictionary is a pair of source and destination arrays.
# Nodes are integer IDs starting from zero. Nodes IDs of different types have
# separate countings.
import dgl
import numpy as np


Using backend: pytorch


DGL supports creating a graph from a variety of data sources. The following
code creates the same graph as the above.

Creating from scipy matrix



In [3]:
import scipy.sparse as sp
# Creating from networkx graph
import networkx as nx


Manipulating heterograph
------------------------
You can create a more realistic heterograph using the ACM dataset. To do this, first 
download the dataset as follows:



In [4]:
import scipy.io
import urllib.request

data_url = 'https://data.dgl.ai/dataset/ACM.mat'
data_file_path = 'ACM.mat'

urllib.request.urlretrieve(data_url, data_file_path)
data = scipy.io.loadmat(data_file_path)
print(list(data.keys()))

['__header__', '__version__', '__globals__', 'TvsP', 'PvsA', 'PvsV', 'AvsF', 'VvsC', 'PvsL', 'PvsC', 'A', 'C', 'F', 'L', 'P', 'T', 'V', 'PvsT', 'CNormPvsA', 'RNormPvsA', 'CNormPvsC', 'RNormPvsC', 'CNormPvsT', 'RNormPvsT', 'CNormPvsV', 'RNormPvsV', 'CNormVvsC', 'RNormVvsC', 'CNormAvsF', 'RNormAvsF', 'CNormPvsL', 'RNormPvsL', 'stopwords', 'nPvsT', 'nT', 'CNormnPvsT', 'RNormnPvsT', 'nnPvsT', 'nnT', 'CNormnnPvsT', 'RNormnnPvsT', 'PvsP', 'CNormPvsP', 'RNormPvsP']


The dataset stores node information by their types: ``P`` for paper, ``A``
for author, ``C`` for conference, ``L`` for subject code, and so on. The relationships
are stored as SciPy sparse matrix under key ``XvsY``, where ``X`` and ``Y``
could be any of the node type code.

The following code prints out some statistics about the paper-author relationships.



In [5]:
print(type(data['PvsA']))
print('#Papers:', data['PvsA'].shape[0])
print('#Authors:', data['PvsA'].shape[1])
print('#Links:', data['PvsA'].nnz)

<class 'scipy.sparse.csc.csc_matrix'>
#Papers: 12499
#Authors: 17431
#Links: 37055


Converting this SciPy matrix to a heterograph in DGL is straightforward.



Create a subset of the ACM graph using the paper-author, paper-paper, 
and paper-subject relationships.  Meanwhile, also add the reverse
relationship to prepare for the later sections.



In [6]:
G = dgl.heterograph({
        ('paper', 'written-by', 'author') : data['PvsA'],
        ('author', 'writing', 'paper') : data['PvsA'].transpose(),
        ('paper', 'citing', 'paper') : data['PvsP'],
        ('paper', 'cited', 'paper') : data['PvsP'].transpose(),
        ('paper', 'is-about', 'subject') : data['PvsL'],
        ('subject', 'has', 'paper') : data['PvsL'].transpose(),
    })
G

Graph(num_nodes={'author': 17431, 'paper': 12499, 'subject': 73},
      num_edges={('paper', 'written-by', 'author'): 37055, ('author', 'writing', 'paper'): 37055, ('paper', 'citing', 'paper'): 30789, ('paper', 'cited', 'paper'): 30789, ('paper', 'is-about', 'subject'): 12499, ('subject', 'has', 'paper'): 12499},
      metagraph=[('author', 'paper'), ('paper', 'author'), ('paper', 'paper'), ('paper', 'paper'), ('paper', 'subject'), ('subject', 'paper')])

**Metagraph** (or network schema) is a useful summary of a heterograph.
Serving as a template for a heterograph, it tells how many types of objects
exist in the network and where the possible links exist.

DGL provides easy access to the metagraph, which could be visualized using
external tools.



In [7]:
import matplotlib.pyplot as plt


Learning tasks associated with heterographs
-------------------------------------------
Some of the typical learning tasks that involve heterographs include:

* *Node classification and regression* to predict the class of each node or
  estimate a value associated with it.

* *Link prediction* to predict if there is an edge of a certain
  type between a pair of nodes, or predict which other nodes a particular
  node is connected with (and optionally the edge types of such connections).

* *Graph classification/regression* to assign an entire
  heterograph into one of the target classes or to estimate a numerical
  value associated with it.

In this tutorial, we designed a simple example for the first task.

A semi-supervised node classification example
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Our goal is to predict the publishing conference of a paper using the ACM
academic graph we just created. To further simplify the task, we only focus
on papers published in three conferences: *KDD*, *ICML*, and *VLDB*. All
the other papers are not labeled, making it a semi-supervised setting.

The following code extracts those papers from the raw dataset and prepares 
the training, validation, testing split.

Note:  in this version we look at 4 conferences: SOSP, SODA, Sigcom and VLDB.

In [83]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

pvc = data['PvsC'].tocsr()
#   sosp = 7
#   soda = 5
#   sigcom =  9
#   vldb = 13
c_selected = [7, 5, 9, 13]  

p_selected = pvc[:, c_selected].tocoo()
# remake 7,5,9,13 labels as 0,1,2,3
labels = pvc.indices
labels[labels==0] = 13
labels[labels==7] = 0
labels[labels==1] = 13
labels[labels == 5] = 1
labels[labels==2] = 13
labels[labels == 9]= 2
labels[labels == 3] = 13
labels[labels == 13] = 3
labels = torch.tensor(labels).long()

# generate train/val/test split
pid = p_selected.row
shuffle = np.random.permutation(pid)

train_idx = torch.tensor(shuffle[0:1400]).long()
val_idx = torch.tensor(shuffle[1400:1500]).long()
test_idx = torch.tensor(shuffle[1500:]).long()

In [84]:
print( len(labels[labels==0]), len(labels[labels==1]), len(labels[labels==2]))

332 662 648


In [85]:
print(len(train_idx), len(test_idx))

1400 719


Relational-GCN on heterograph
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
We use `Relational-GCN <https://arxiv.org/abs/1703.06103>`_ to learn the
representation of nodes in the graph. Its message-passing equation is as
follows:
for each message type we will have a fully connected layer of trainable parameters W. Then for 
for each node and 
each edge type we compute

\begin{align} \sum_{j\in\mathcal{N}_r(i)} W_r^{(l)}h_j^{(l)} \end{align}

following that sum over each edge type and apply sigma

\begin{align}h_i^{(l+1)} = \sigma\left(\sum_{r\in \mathcal{R}}
   \sum_{j\in\mathcal{N}_r(i)}W_r^{(l)}h_j^{(l)}\right)\end{align}

Breaking down the equation, you see that there are two parts in the
computation.

(i) Message computation and aggregation within each relation $r$

(ii) Reduction that merges the results from multiple relationships

Following this intuition, perform message passing on a heterograph in
two steps.

(i) Per-edge-type message passing

(ii) Type wise reduction



In [86]:
import dgl.function as fn

class HeteroRGCNLayer(nn.Module):
    def __init__(self, in_size, out_size, etypes):
        super(HeteroRGCNLayer, self).__init__()
        # W_r for each relation
        self.weight = nn.ModuleDict({
                name : nn.Linear(in_size, out_size) for name in etypes
            })

    def forward(self, G, feat_dict):
        # The input is a dictionary of node features for each type
        funcs = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            # Compute W_r * h
            Wh = self.weight[etype](feat_dict[srctype])
            # Save it in graph for message passing
            G.nodes[srctype].data['Wh_%s' % etype] = Wh
            # Specify per-relation message passing functions: (message_func, reduce_func).
            # Note that the results are saved to the same destination feature 'h', which
            # hints the type wise reducer for aggregation.
            funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
        # Trigger message passing of multiple types.
        # The first argument is the message passing functions for each relation.
        # The second one is the type wise reducer, could be "sum", "max",
        # "min", "mean", "stack"
        G.multi_update_all(funcs, 'sum')
        # return the updated node feature dictionary
        return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}

Create a simple GNN by stacking two ``HeteroRGCNLayer``. Since the
nodes do not have input features, make their embeddings trainable.



In [87]:
G.canonical_etypes

[('paper', 'written-by', 'author'),
 ('author', 'writing', 'paper'),
 ('paper', 'citing', 'paper'),
 ('paper', 'cited', 'paper'),
 ('paper', 'is-about', 'subject'),
 ('subject', 'has', 'paper')]

In [88]:
G

Graph(num_nodes={'author': 17431, 'paper': 12499, 'subject': 73},
      num_edges={('paper', 'written-by', 'author'): 37055, ('author', 'writing', 'paper'): 37055, ('paper', 'citing', 'paper'): 30789, ('paper', 'cited', 'paper'): 30789, ('paper', 'is-about', 'subject'): 12499, ('subject', 'has', 'paper'): 12499},
      metagraph=[('author', 'paper'), ('paper', 'author'), ('paper', 'paper'), ('paper', 'paper'), ('paper', 'subject'), ('subject', 'paper')])

In [89]:
G.ntypes

['author', 'paper', 'subject']

In [90]:
embed_dict = {ntype : nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), 10))
              for ntype in G.ntypes}
for key, embed in embed_dict.items():
    nn.init.xavier_uniform_(embed)
embed = nn.ParameterDict(embed_dict)
embed

ParameterDict(
    (author): Parameter containing: [torch.FloatTensor of size 17431x10]
    (paper): Parameter containing: [torch.FloatTensor of size 12499x10]
    (subject): Parameter containing: [torch.FloatTensor of size 73x10]
)

In [91]:
class HeteroRGCN(nn.Module):
    def __init__(self, G, in_size, hidden_size, out_size):
        super(HeteroRGCN, self).__init__()
        # Use trainable node embeddings as featureless inputs.
        embed_dict = {ntype : nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), in_size))
                      for ntype in G.ntypes}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        # create layers
        self.layer1 = HeteroRGCNLayer(in_size, hidden_size, G.etypes)
        self.layer2 = HeteroRGCNLayer(hidden_size, out_size, G.etypes)

    def forward(self, G):
        h_dict = self.layer1(G, self.embed)
        h_dict = {k : F.leaky_relu(h) for k, h in h_dict.items()}
        h_dict = self.layer2(G, h_dict)
        # get paper logits
        return h_dict['paper']

Train and evaluate
~~~~~~~~~~~~~~~~~~
Train and evaluate this network.



In [92]:
# Create the model. The output has four logits for four classes.
model = HeteroRGCN(G, 10, 10, 4)

opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

best_val_acc = 0
best_test_acc = 0

for epoch in range(100):
    logits = model(G)
    # The loss is computed only for labeled nodes.
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])

    pred = logits.argmax(1)
    train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
    val_acc = (pred[val_idx] == labels[val_idx]).float().mean()
    test_acc = (pred[test_idx] == labels[test_idx]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc

    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch % 5 == 0:
        print('Loss %.4f, Train Acc %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
            loss.item(),
            train_acc.item(),
            val_acc.item(),
            best_val_acc.item(),
            test_acc.item(),
            best_test_acc.item(),
        ))

Loss 1.4772, Train Acc 0.2371, Val Acc 0.2300 (Best 0.2300), Test Acc 0.2281 (Best 0.2281)
Loss 1.2481, Train Acc 0.6450, Val Acc 0.5300 (Best 0.5300), Test Acc 0.5410 (Best 0.5410)
Loss 1.0722, Train Acc 0.7607, Val Acc 0.5800 (Best 0.5800), Test Acc 0.6203 (Best 0.6203)
Loss 0.8221, Train Acc 0.8436, Val Acc 0.7400 (Best 0.7500), Test Acc 0.7955 (Best 0.7747)
Loss 0.5720, Train Acc 0.8893, Val Acc 0.8100 (Best 0.8100), Test Acc 0.8150 (Best 0.8150)
Loss 0.3809, Train Acc 0.9586, Val Acc 0.8600 (Best 0.8600), Test Acc 0.8428 (Best 0.8428)
Loss 0.2482, Train Acc 0.9857, Val Acc 0.8700 (Best 0.8700), Test Acc 0.8707 (Best 0.8554)
Loss 0.1638, Train Acc 0.9929, Val Acc 0.8500 (Best 0.8700), Test Acc 0.8790 (Best 0.8554)
Loss 0.1139, Train Acc 0.9950, Val Acc 0.8800 (Best 0.8800), Test Acc 0.8971 (Best 0.8971)
Loss 0.0848, Train Acc 0.9964, Val Acc 0.8900 (Best 0.8900), Test Acc 0.9068 (Best 0.8985)
Loss 0.0661, Train Acc 0.9986, Val Acc 0.8900 (Best 0.8900), Test Acc 0.9068 (Best 0.8985)

In [93]:
logits = model(G)
pred = logits.argmax(1)
test_acc = (pred[test_idx] == labels[test_idx]).float().mean()
print(test_acc)

tensor(0.8707)


In [94]:
x = data['C'][:]
print(x)
len(x)

[[array(['KDD'], dtype='<U3')]
 [array(['SIGMOD'], dtype='<U6')]
 [array(['WWW'], dtype='<U3')]
 [array(['SIGIR'], dtype='<U5')]
 [array(['CIKM'], dtype='<U4')]
 [array(['SODA'], dtype='<U4')]
 [array(['STOC'], dtype='<U4')]
 [array(['SOSP'], dtype='<U4')]
 [array(['SPAA'], dtype='<U4')]
 [array(['SIGCOMM'], dtype='<U7')]
 [array(['MobiCOMM'], dtype='<U8')]
 [array(['ICML'], dtype='<U4')]
 [array(['COLT'], dtype='<U4')]
 [array(['VLDB'], dtype='<U4')]]


14

try this:
   sosp = 7
   soda = 5
   sigcom =  9
   vldb = 13

In [95]:
mat = np.zeros([4,4])
mat

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [97]:
mat = np.zeros([4,4])
for i in train_idx:
    if labels[i] == 0:
        mat[0, pred[i]]+=1
    if labels[i] == 1:
        mat[1, pred[i]]+=1
    if labels[i] == 2:
        mat[2, pred[i]]+=1
    if labels[i] == 3:
        mat[3, pred[i]]+=1
mat

array([[200.,   0.,   0.,   0.],
       [  0., 421.,   0.,   0.],
       [  0.,   0., 415.,   0.],
       [  0.,   0.,   0., 364.]])

In [98]:
mat = np.zeros([4,4])
for i in test_idx:
    if labels[i] == 0:
        mat[0, pred[i]]+=1
    if labels[i] == 1:
        mat[1, pred[i]]+=1
    if labels[i] == 2:
        mat[2, pred[i]]+=1
    if labels[i] == 3:
        mat[3, pred[i]]+=1
mat

array([[ 92.,   2.,  12.,   8.],
       [  2., 206.,   1.,   5.],
       [  8.,  28., 161.,  10.],
       [  1.,  16.,   0., 167.]])

In [99]:
for i in range(4):
    tot = sum(mat[i,:])
    mat[i,:] = np.round(100.0*mat[i,:]/tot,0)
mat

array([[81.,  2., 11.,  7.],
       [ 1., 96.,  0.,  2.],
       [ 4., 14., 78.,  5.],
       [ 1.,  9.,  0., 91.]])

In [101]:
import pandas as pd
df = pd.DataFrame(mat, index =['sosp', 'soda', 'sigcom','vldb'])
df

,0,1,2,3
sosp,81.0,2.0,11.0,7.0
soda,1.0,96.0,0.0,2.0
sigcom,4.0,14.0,78.0,5.0
vldb,1.0,9.0,0.0,91.0


What's next?
------------
* Check out our full implementation in PyTorch
  `here <https://github.com/dmlc/dgl/tree/master/examples/pytorch/rgcn-hetero>`_.

* We also provide the following model examples:

  * `Graph Convolutional Matrix Completion <https://arxiv.org/abs/1706.02263>_`,
    which we implement in MXNet
    `here <https://github.com/dmlc/dgl/tree/v0.4.0/examples/mxnet/gcmc>`_.

  * `Heterogeneous Graph Attention Network <https://arxiv.org/abs/1903.07293>`_
    requires transforming a heterograph into a homogeneous graph according to
    a given metapath (i.e. a path template consisting of edge types).  We
    provide :func:`dgl.transform.metapath_reachable_graph` to do this.  See full
    implementation
    `here <https://github.com/dmlc/dgl/tree/master/examples/pytorch/han>`_.

  * `Metapath2vec <https://dl.acm.org/citation.cfm?id=3098036>`_ requires
    generating random walk paths according to a given metapath.  Please
    refer to the full metapath2vec implementation
    `here <https://github.com/dmlc/dgl/tree/master/examples/pytorch/metapath2vec>`_.

* :doc:`Full heterograph API reference <../../api/python/heterograph>`.

